In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#The aim is to predict which customer will click or not

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import pandas as pd

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# function for comparing different approaches



/kaggle/input/int-tbl-clks/interactionstbl_features_clicks.csv
/kaggle/input/intclick4/interactionstbl_features_clicks2.csv
/kaggle/input/interactionstbl-features-clicks/interactionstbl_features_clicks.csv
/kaggle/input/pegamktcomms/export_mkt_comm.csv
/kaggle/input/exercise-introduction/__results__.html
/kaggle/input/exercise-introduction/submission.csv
/kaggle/input/exercise-introduction/__notebook__.ipynb
/kaggle/input/exercise-introduction/__output__.json
/kaggle/input/exercise-introduction/custom.css
/kaggle/input/intclick2/interactionstbl_features_clicks2.csv
/kaggle/input/intclick3/interactionstbl_features_clicks2.csv
/kaggle/input/interactiontbl-features/interactionstbl_features.csv


In [2]:
interactiontbl_file_path = '/kaggle/input/intclick4/interactionstbl_features_clicks2.csv'
train_interaction_data = pd.read_csv(interactiontbl_file_path) 
train_interaction_data.columns

Index(['userid', 'interactionid', 'propid', 'dateandtime', 'proprules',
       'response', 'response_click', 'isimpression',
       'TotalCreditBalanceSuffix', 'ExternalCreditCardAccountBalanceSuffix',
       'NetPositionSuffix', 'TotalDebitBalance', 'NetPosition',
       'CurrentAccountBalanceSuffix', 'CurrentSavingsAccounBalanceSuffix',
       'TotalCreditBalance', 'EventId', 'CurrentISASavingsBalance',
       'CurrentSavingsBalance', 'ISAAnnualLimit', 'CLV',
       'ExternalCreditCardBalance', 'AccountOpenDate', 'ATMLastUsedDate',
       'Income', 'Avg6mnthBalance', 'CustomerValue', 'CreditTurnoverAmount',
       'Customer_ID', 'age', 'ATMLoginCounter', 'Sex', 'AccountType',
       'AccNoCurrent', 'ISAHeadroom1', 'ZipCode', 'CustomerName', 'Title',
       'OnlineUsageCount', 'CurrentMortgageBalance', 'Case_ID',
       'AccNoMastercard', 'AverageRevenue', 'AccNoSavings', 'CurrentBalance',
       'ISABenefits1', 'CustomerMobileNo', 'CustomerEmailAddress',
       'TotalDebitBalanceSuff

In [3]:
# Remove rows with missing target, separate target from predictors
train_interaction_data.dropna(axis=0, subset=['response_click'], inplace=True)
train_interaction_data.head()

,userid,interactionid,propid,dateandtime,proprules,response,response_click,isimpression,TotalCreditBalanceSuffix,ExternalCreditCardAccountBalanceSuffix,...,AverageRevenue,AccNoSavings,CurrentBalance,ISABenefits1,CustomerMobileNo,CustomerEmailAddress,TotalDebitBalanceSuffix,Channel,breuserid,brecontext
0,1,2.020000e+13,m1.2.onl_rhb.1,17/08/2020 12:16,1,click,1,True,CR,DR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
1,1,2.020000e+13,m1.2.onl_box.1,17/08/2020 12:16,1,NaN,0,True,CR,DR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
2,1,2.020000e+13,m1.2.onl_ban.2,17/08/2020 12:16,1,NaN,0,True,CR,DR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
3,2,2.020000e+13,m1.2.onl_box.1,17/08/2020 12:16,1,click,1,True,CR,DR,...,75,82-63-24 90658769,456,300,79402654208,phil.ae@pega.com,DR,web,aetuk,all
4,2,2.020000e+13,m1.2.onl_ban.2,17/08/2020 12:16,1,NaN,0,True,CR,DR,...,75,82-63-24 90658769,456,300,79402654208,phil.ae@pega.com,DR,web,aetuk,all


In [4]:
target= train_interaction_data.response_click

In [5]:
#y = interaction_data.response
target.head()

0    1
1    0
2    0
3    1
4    0
Name: response_click, dtype: int64

In [6]:
# To keep things simple, we'll drop columns with missing values
cols_with_missing = [col for col in train_interaction_data.columns 
                     if train_interaction_data[col].isnull().any()] 
print("cols_with_missing:",cols_with_missing)

cols_with_missing: ['response', 'isimpression']


In [7]:
candidate_train_predictors = train_interaction_data.drop(['interactionid', 'response_click'] + cols_with_missing, axis=1)
candidate_test_predictors = train_interaction_data.drop(['interactionid'] + cols_with_missing, axis=1)
candidate_train_predictors.head()

,userid,propid,dateandtime,proprules,TotalCreditBalanceSuffix,ExternalCreditCardAccountBalanceSuffix,NetPositionSuffix,TotalDebitBalance,NetPosition,CurrentAccountBalanceSuffix,...,AverageRevenue,AccNoSavings,CurrentBalance,ISABenefits1,CustomerMobileNo,CustomerEmailAddress,TotalDebitBalanceSuffix,Channel,breuserid,brecontext
0,1,m1.2.onl_rhb.1,17/08/2020 12:16,1,CR,DR,CR,23,9000,CR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
1,1,m1.2.onl_box.1,17/08/2020 12:16,1,CR,DR,CR,23,9000,CR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
2,1,m1.2.onl_ban.2,17/08/2020 12:16,1,CR,DR,CR,23,9000,CR,...,57,82-63-24 90658765,787,300,79402654208,phil.nicholson@pega.com,DR,web,aetuk,all
3,2,m1.2.onl_box.1,17/08/2020 12:16,1,CR,DR,CR,0,235,CR,...,75,82-63-24 90658769,456,300,79402654208,phil.ae@pega.com,DR,web,aetuk,all
4,2,m1.2.onl_ban.2,17/08/2020 12:16,1,CR,DR,CR,0,235,CR,...,75,82-63-24 90658769,456,300,79402654208,phil.ae@pega.com,DR,web,aetuk,all


In [8]:
# "cardinality" means the number of unique values in a column.
# We use it as our only way to select categorical columns here. This is convenient, though
# a little arbitrary.
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 100 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

train_predictors.head()
test_predictors.head()

,userid,propid,TotalCreditBalanceSuffix,ExternalCreditCardAccountBalanceSuffix,NetPositionSuffix,CurrentAccountBalanceSuffix,CurrentSavingsAccounBalanceSuffix,EventId,AccountOpenDate,ATMLastUsedDate,...,ATMLoginCounter,ISAHeadroom1,OnlineUsageCount,CurrentMortgageBalance,Case_ID,AccNoMastercard,AverageRevenue,CurrentBalance,ISABenefits1,CustomerMobileNo
0,1,m1.2.onl_rhb.1,CR,DR,CR,CR,CR,e1,20110223T050000.000 GMT,20130515T040000.000 GMT,...,1,12000,1,-170000,1,5.430000e+15,57,787,300,79402654208
1,1,m1.2.onl_box.1,CR,DR,CR,CR,CR,e1,20110223T050000.000 GMT,20130515T040000.000 GMT,...,1,12000,1,-170000,1,5.430000e+15,57,787,300,79402654208
2,1,m1.2.onl_ban.2,CR,DR,CR,CR,CR,e1,20110223T050000.000 GMT,20130515T040000.000 GMT,...,1,12000,1,-170000,1,5.430000e+15,57,787,300,79402654208
3,2,m1.2.onl_box.1,CR,DR,CR,CR,CR,e1,20110223T050000.000 GMT,20130515T040000.000 GMT,...,1,12000,1,-110000,2,5.930000e+15,75,456,300,79402654208
4,2,m1.2.onl_ban.2,CR,DR,CR,CR,CR,e1,20110223T050000.000 GMT,20130515T040000.000 GMT,...,1,12000,1,-110000,2,5.930000e+15,75,456,300,79402654208


In [9]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals = get_mae(predictors_without_categoricals, target)

mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, target)

print('Mean Absolute Error when Dropping Categoricals: ' + str(int(mae_without_categoricals)))
print('Mean Abslute Error with One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

Mean Absolute Error when Dropping Categoricals: 0
Mean Abslute Error with One-Hot Encoding: 0


In [11]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)

In [12]:
final_test.head()

,proprules,TotalDebitBalance,NetPosition,TotalCreditBalance,CurrentISASavingsBalance,CurrentSavingsBalance,ISAAnnualLimit,CLV,ExternalCreditCardBalance,Income,...,AccNoSavings_82-63-24 96658795,AccNoSavings_82-63-24 96658796,AccNoSavings_82-63-24 96658797,AccNoSavings_82-63-24 96658798,CustomerEmailAddress_phil.ae@pega.com,CustomerEmailAddress_phil.nicholson@pega.com,TotalDebitBalanceSuffix_DR,Channel_web,breuserid_aetuk,brecontext_all
0,1,23,9000,9000,5000,8000,12000,7,45345,40000,...,0,0,0,0,0,1,1,1,1,1
1,1,23,9000,9000,5000,8000,12000,7,45345,40000,...,0,0,0,0,0,1,1,1,1,1
2,1,23,9000,9000,5000,8000,12000,7,45345,40000,...,0,0,0,0,0,1,1,1,1,1
3,1,0,235,29000,15000,9000,12000,4,346346,4757,...,0,0,0,0,1,0,1,1,1,1
4,1,0,235,29000,15000,9000,12000,4,346346,4757,...,0,0,0,0,1,0,1,1,1,1
